In [31]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from pys.data_collection.market_data import run_pipeline_market
from pys.data_collection.fundamental_data import run_pipeline_fundamental
from pys.data_collection.tech_analysis import run_pipeline_technical
from pys.data_collection.data_integration import run_pipeline_integration
from pys.data_collection.bonds_processor import run_pipeline_bonds_processor
from pys.data_collection.market_cap import run_pipeline_market_cap
from pys.data_collection.bonds_parser import run_bond_pipeline
from pys.data_collection.bonds_kbd_pipeline import run_bond_selection_with_kbd
from pys.porfolio_optimization.signal_generator import run_pipeline_signal_generator

from pys.data_collection.news_pipeline import NewsPipeline
from pys.porfolio_optimization.portfolio_optimizer import PortfolioOptimizer, run_all_optimization_models
from pys.porfolio_optimization.backtester import Backtester


from pys.data_collection.private_info import BASE_PATH
from pys.utils.run_with_config import run_with_config

## Globals

In [33]:
from pys.data_collection.private_info import token, YOUR_API_ID, YOUR_API_HASH, BASE_PATH

kursach_directory_path = BASE_PATH

tickers = pd.read_csv(f"{kursach_directory_path}/data/meta/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2024-01-01"
end_date="2025-04-15"
timeframe="1h"

overall_df = f'{kursach_directory_path}/data/df.csv'

## Парсинг и пердобработка

In [104]:
run_pipeline_market(
    tickers=tickers_list,
    start_date=start_date,
    end_date=end_date,
    token=token
)

2025-04-26 18:45:01 - MarketDataPipeline - INFO - MarketDataPipeline initialized with base directory: //Users/aeshef/Documents/GitHub/kursach/data
2025-04-26 18:45:01 - MarketDataPipeline - INFO - Starting market data pipeline for 16 tickers
2025-04-26 18:45:01 - MarketDataPipeline - INFO - Date range: 2024-01-01 to 2025-04-15, timeframe: 1min
2025-04-26 18:45:01 - TinkoffDataDownloader - INFO - Initializing instruments from Tinkoff API
2025-04-26 18:45:05 - TinkoffDataDownloader - INFO - Successfully loaded 6224 instruments, 794 in RUB
2025-04-26 18:45:05 - MarketDataManager - INFO - MarketDataManager initialized
2025-04-26 18:45:05 - MarketDataPipeline - INFO - Starting process: Data download
2025-04-26 18:45:05 - MarketDataManager - INFO - Getting data for 16 tickers with 5 workers
2025-04-26 18:45:05 - MarketDataManager - INFO - Getting data for SBER from 2024-01-01 to 2025-04-15 with timeframe 1min
2025-04-26 18:45:05 - MarketDataManager - INFO - Getting data for GAZP from 2024-01

{'SBER':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  268.87  271.04  271.04  268.87      17
 1      2024-01-02 07:17:00  268.87  271.00  271.07  268.87     194
 2      2024-01-02 07:18:00  269.00  270.83  271.00  269.00      94
 3      2024-01-02 07:19:00  269.91  269.91  270.83  269.91      66
 4      2024-01-02 07:20:00  269.91  270.72  270.76  269.91     102
 ...                    ...     ...     ...     ...     ...     ...
 412933 2025-04-14 20:45:00  296.45  296.36  296.46  296.36    1464
 412934 2025-04-14 20:46:00  296.36  296.49  296.49  296.36    1395
 412935 2025-04-14 20:47:00  296.49  296.48  296.50  296.48    1327
 412936 2025-04-14 20:48:00  296.49  296.55  296.55  296.48    1020
 412937 2025-04-14 20:49:00  296.54  296.50  296.55  295.83    8854
 
 [412938 rows x 6 columns],
 'GAZP':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  160.34  159.40  160.34  159.40      22
 1

In [105]:
market_caps_df = run_pipeline_market_cap(
    base_path=BASE_PATH,
    tickers=tickers_list
)

2025-04-26 18:52:09 - MarketCapParser - INFO - Запуск пайплайна парсинга капитализаций для выбранных тикеров


2025-04-26 18:52:09 - MarketCapParser - INFO - Запрос данных о капитализации с MOEX API для выбранных тикеров
2025-04-26 18:52:09 - MarketCapParser - INFO - Получены данные о капитализации для 16 компаний
2025-04-26 18:52:09 - MarketCapParser - INFO - Общий файл с капитализациями сохранен в /Users/aeshef/Documents/GitHub/kursach/data/all_caps.csv
2025-04-26 18:52:09 - MarketCapParser - INFO - Пайплайн парсинга капитализаций завершен успешно. Обработано 16 компаний.


## Fundamental

In [107]:
run_pipeline_fundamental(ticker_list=tickers_list)

2025-04-26 18:52:18 - FundamentalPipeline - INFO - Processing tickers: ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL']
2025-04-26 18:52:18 - FundamentalPipeline - INFO - Fetching data for SBER
2025-04-26 18:52:18 - FundamentalPipeline - INFO - Successfully processed SBER
2025-04-26 18:52:18 - FundamentalPipeline - INFO - Fetching data for GAZP
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Successfully processed GAZP
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Fetching data for LKOH
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Successfully processed LKOH
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Fetching data for GMKN
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Successfully processed GMKN
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Fetching data for ROSN
2025-04-26 18:52:19 - FundamentalPipeline - INFO - Successfully processed ROSN
2025-04-26 18:52:19 - FundamentalPi

## Tech

In [ ]:
run_pipeline_technical(tickers=tickers_list)

2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-26 13:43:14 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-26 13:43:14 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_tech_indicators.csv
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Сохранен график цены с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_price_indicators.png


Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-26 13:43:14 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-26 13:43:14 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_tech_indicators.csv
2025-04-26 13:43:14 - TechAnalysisPipeline - INFO - Сохранен график цены с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_price_indicators.png


Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-26 13:43:15 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-26 13:43:15 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-26 13:43:15 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-26 13:43:15 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-26 13:43:15 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-26 13:43:15 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/LKOH/tech_analysis/LKOH_tech_indicators.csv
2025-04-26 13:43:15 - TechAnalysisPipeline - INFO - Сохранен график цены с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/LKOH/tech_analysis/LKOH_price_indicators.png


...

2025-04-26 13:43:18 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-26 13:43:18 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-26 13:43:18 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-26 13:43:18 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-26 13:43:18 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-26 13:43:18 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_tech_indicators.csv
2025-04-26 13:43:18 - TechAnalysisPipeline - INFO - Сохранен график цены с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_price_indicators.png


Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-26 13:43:19 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-26 13:43:19 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_tech_indicators.csv
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Сохранен график цены с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_price_indicators.png


Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/summaries/tech_analysis_summary.txt
2025-04-26 13:43:19 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===


## News

In [ ]:
%run -i ../pys/utils/monkey_patch.py
apply_patch()

run_with_config(
    script_path=f"{kursach_directory_path}/pys/data_collection/news_pipeline.py",
    class_or_function="NewsPipeline",
    method="run_pipeline",
    config_filename="news_pipeline_config.json",
    
    tickers=tickers_list,
    collect_telegram=True,
    telegram_api_id=YOUR_API_ID,
    telegram_api_hash=YOUR_API_HASH,
    telegram_channel="cbrstocks",
    telegram_limit=10000,
    start_date=start_date,
    end_date=end_date,
    use_cached_telegram=True
)


Конфигурация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/meta/news_pipeline_config.json
Запуск команды: /Users/aeshef/Documents/GitHub/kursach/.venv/bin/python /Users/aeshef/Documents/GitHub/kursach/pys/data_collection/news_pipeline.py --config /Users/aeshef/Documents/GitHub/kursach/data/meta/news_pipeline_config.json
Monkey patch applied: inspect.getargspec added
Запуск pipeline.run_pipeline с параметрами: {'tickers': ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL'], 'collect_telegram': True, 'telegram_api_id': 28994010, 'telegram_api_hash': '6e7a57fdfe1a10b0a3434104b42badf2', 'telegram_channel': 'cbrstocks', 'telegram_limit': 10000, 'start_date': datetime.date(2024, 1, 1), 'end_date': datetime.date(2025, 4, 15), 'use_cached_telegram': True}
2025-04-26 13:43:21 - NewsPipeline - INFO - Запуск пайплайна анализа новостей для 16 тикеров
2025-04-26 13:43:21 - NewsPipeline - INFO - Диапазон дат: 2024

0

In [ ]:
run_pipeline_integration(tickers=tickers_list)

2025-04-26 13:46:38 - DataIntegrator - INFO - Обработка тикера SBER...
2025-04-26 13:46:38 - DataIntegrator - INFO - Данные для SBER загружены за период: 2024-01-02 - 2025-04-14
2025-04-26 13:46:38 - DataIntegrator - INFO -   - Обнаружено 5480 пропущенных значений для SBER
2025-04-26 13:46:38 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-26 13:46:38 - DataIntegrator - INFO - Обработка тикера GAZP...
2025-04-26 13:46:38 - DataIntegrator - INFO - Данные для GAZP загружены за период: 2024-01-02 - 2025-04-14
2025-04-26 13:46:38 - DataIntegrator - INFO -   - Обнаружено 5423 пропущенных значений для GAZP
2025-04-26 13:46:38 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-26 13:46:38 - DataIntegrator - INFO - Обработка тикера LKOH...
2025-04-26 13:46:38 - DataIntegrator - INFO - Данные для LKOH загружены за период: 2024-01-02 - 2025-04-14
2025-04-26 13:46:38 - DataIntegrator - INFO -   - Обнаружено 5427 пропущенных значений

In [34]:
from pys.data_collection.private_info import BASE_PATH
from pys.data_collection.kbd import load_kbd_data

kbd_data = load_kbd_data(
    base_path=BASE_PATH,
    start_date=start_date,
    end_date=end_date,
    update_data=True
)

2025-04-27 18:13:42 - KBDDownloader - INFO - KBDDownloader initialized
2025-04-27 18:13:42 - KBDDownloader - INFO - Fetching KBD data from 2024-01-01 00:00:00 to 2025-04-15 00:00:00
2025-04-27 18:13:42 - KBDDownloader - INFO - Raw KBD data saved to /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/raw/raw_kbd_data_2024-01-01_to_2025-04-15.csv
2025-04-27 18:13:42 - KBDDownloader - INFO - Raw file contains 261 lines
2025-04-27 18:13:42 - KBDDownloader - INFO - First line contains 'zcyc', skipping it
2025-04-27 18:13:42 - KBDDownloader - INFO - Parsed CSV with columns: ['tradedate', 'tradetime', 'period_0.25', 'period_0.5', 'period_0.75', 'period_1.0', 'period_2.0', 'period_3.0', 'period_5.0', 'period_7.0', 'period_10.0', 'period_15.0', 'period_20.0', 'period_30.0']
2025-04-27 18:13:42 - KBDDownloader - INFO - Successfully parsed dates, first date: 2024-01-03 00:00:00
2025-04-27 18:13:42 - KBDDownloader - INFO - Processed MOEX data: 256 rows with 12 tenors
2025-04-27 18

## Построение


In [35]:
##### 1. Параметры загрузки данных КБД #####
kbd_params = {
    'base_path': BASE_PATH,
    'start_date': '2024-01-01',
    'end_date': '2025-04-15',
    'update_data': True,  # Обновлять ли данные КБД с MOEX
}

##### 2. Параметры основного пайплайна #####
pipeline_params = {
    'base_path': BASE_PATH,
    'name': f"balanced_portfolio_{datetime.now().strftime('%m%d_%H%M')}",
    'strategy_profile': 'moderate',  # 'conservative', 'moderate', 'aggressive'
    'min_position_weight': 0.01,     # Минимальный вес позиции в портфеле
    'min_rf_allocation': 0.3,        # Минимальная доля безрисковых активов
    'max_rf_allocation': 0.5,        # Максимальная доля безрисковых активов
    'risk_free_rate': 0.1,           # Безрисковая ставка доходности
    'max_weight': 0.15,              # Максимальный вес одной позиции
}

##### 3. Параметры для обработки облигаций #####
bond_params = {
    'start_date': '2024-01-01',
    'end_date': '2025-04-15',
    'min_bonds': 5,                  # Минимальное число облигаций
    'max_threshold': 99,             # Максимальный порог непрерывности
    'strategy_profile': 'moderate',  # Профиль риска
    'kbd_yield_adjustment': -2.0,    # Корректировка доходности КБД
    'update_kbd_data': False,        # Не обновлять КБД данные
    'excluded_issuers': ['ВТБ', 'Мечел'],  # Исключаемые эмитенты
    'n_bonds': 5,                    # Целевое число облигаций
    
    # Не передаются напрямую в process_bond_pipeline, но используются в run_bond_selection_with_kbd
    'portfolio_stability': 0.7,      # Стабильность портфеля (0.0-1.0)
    'use_kbd_recommendations': True, # Использовать рекомендации КБД
    'kbd_duration_flexibility': 1.5, # Множитель диапазона дюрации
    'max_adjustment_iterations': 3,  # Макс. итераций адаптивной настройки
    'weighting_strategy': None,      # Стратегия взвешивания (None для автовыбора)
    'override_params': None          # Ручное переопределение параметров
}

##### 4. Параметры для генерации сигналов #####
signal_params = {
    'weight_tech': 0.5,              # Вес технических сигналов
    'weight_sentiment': 0.3,         # Вес сентимент-сигналов
    'weight_fundamental': 0.2,       # Вес фундаментальных сигналов
    'threshold_buy': 0.5,            # Порог для сигнала покупки
    'threshold_sell': -0.5,          # Порог для сигнала продажи
    'top_pct': 0.3,                  # Процент лучших акций для shortlist
    'save_ticker_visualizations': False  # Сохранять визуализации по тикерам
}

##### 5. Параметры для стандартного портфеля #####
standard_portfolio_params = {
    'risk_free_rate': 0.1,
    'min_rf_allocation': 0.3,
    'max_rf_allocation': 0.5,
    'max_weight': 0.15,
    'include_short_selling': False
}

##### 6. Параметры для портфеля с короткими позициями #####
short_portfolio_params = {
    'risk_free_rate': 0.1,
    'train_period': ('2024-01-01', '2024-12-31'),
    'test_period': ('2025-01-01', '2025-04-15'),
    'best_params_file': None,        # Путь к лучшим параметрам из Grid Search
    'verify_with_honest_backtest': True
}

##### 7. Параметры для комбинированного портфеля #####
combined_portfolio_params = {
    'risk_free_rate': 0.1,
    'min_rf_allocation': 0.3,
    'max_rf_allocation': 0.5,
    'max_weight': 0.15,
    'long_ratio': 0.7,               # Соотношение длинных/коротких позиций
    'include_short_selling': True
}

##### 8. Параметры оптимизации портфеля #####
optimization_params = {
    'optimization': 'markowitz',      # 'markowitz' или 'black_litterman'
    'tau': 0.05,                      # Параметр неуверенности для Black-Litterman
    'views': None,                    # Субъективные прогнозы
    'view_confidences': None,         # Уверенность в прогнозах
    'market_caps': None               # Рыночные капитализации
}

##### 9. Параметры для честного бэктеста #####
backtest_params = {
    'train_period': ('2024-01-01', '2024-12-31'),
    'test_period': ('2025-01-01', '2025-04-15'),
    'risk_free_rate': 0.075,
    'use_grid_search_params': True
}

In [36]:
from pys.porfolio_optimization.executor import PipelineExecutor

executor = PipelineExecutor(
    base_path=pipeline_params['base_path'],
    name=pipeline_params['name'],
    strategy_profile=pipeline_params['strategy_profile'],
    min_position_weight=pipeline_params['min_position_weight'],
    min_rf_allocation=pipeline_params['min_rf_allocation'],
    max_rf_allocation=pipeline_params['max_rf_allocation'],
    risk_free_rate=pipeline_params['risk_free_rate'],
    max_weight=pipeline_params['max_weight']
)


2025-04-27 18:13:46 - PipelineExecutor - INFO - Создана структура директорий для запуска run_20250427_181346_balanced_portfolio_0427_1813


In [37]:


# 3. Запускаем пайплайн обработки облигаций (включает обе части - парсинг и выбор)
bond_results = executor.process_bond_pipeline(
    start_date=bond_params['start_date'],
    end_date=bond_params['end_date'],
    min_bonds=bond_params['min_bonds'],
    max_threshold=bond_params['max_threshold'],
    strategy_profile=bond_params['strategy_profile'],
    kbd_yield_adjustment=bond_params['kbd_yield_adjustment'],
    update_kbd_data=bond_params['update_kbd_data'],
    excluded_issuers=bond_params['excluded_issuers'],
    n_bonds=bond_params['n_bonds'],
    kbd_data=kbd_data,
    portfolio_stability=0.7,
    use_kbd_recommendations=True,
    kbd_duration_flexibility=1.5,
    max_adjustment_iterations=3,
    weighting_strategy=None,
    override_params=None,
    output_format='all'
)

2025-04-27 18:13:48 - PipelineExecutor - INFO - Запуск полного цикла обработки облигаций


2025-04-27 18:13:48 - PipelineExecutor - INFO - Создана структура директорий для облигаций в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds
2025-04-27 18:13:48 - MOEXBondHistoricalParser - INFO - MOEX Bond Parser initialized
2025-04-27 18:13:48 - PipelineExecutor - INFO - Датасет облигаций скопирован в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/bonds_dataset.csv
2025-04-27 18:13:48 - PipelineExecutor - INFO - Обработка облигаций: оптимальный порог 99%
2025-04-27 18:13:48 - BondsKBDPipeline - INFO - Запуск пайплайна выбора облигаций с учетом КБД
2025-04-27 18:13:48 - BondsKBDPipeline - INFO - Используем предоставленные данные КБД: 256 записей
2025-04-27 18:13:48 - BondsKBDPipeline - INFO - Данные КБД доступны: 256 записей
2025-04-27 18:13:48 - BondsKBDPipeline - INFO - Копия данных КБД сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_da

Running bond analysis for period 2024-01-01 to 2025-04-15
Found existing dataset: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/period_2024-01-01_2025-04-15/yieldrange_0_100__pricerange_0_200__durationrange_0_360__volthresh_2000__bondvolthresh_60000__known_coupons/period_data.csv
Analyzing bond data continuity...
Threshold 99%: 14 bonds available
Optimal threshold: 99%
Created filtered dataset with 14 bonds and 469 dates


2025-04-27 18:13:48 - KBDAnalyzer - INFO - КБД визуализация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/viz/kbd_curve_20250427_181348.png
2025-04-27 18:13:48 - KBDAnalyzer - INFO - Сформированы метрики КБД: {'date': Timestamp('2024-12-30 00:00:00'), 'market_state': 'inverted', 'optimal_duration': (0.1, 2.0), 'yield_threshold': np.float64(21.58), 'kbd_values': {'0.5Y': np.float64(18.75), '0.75Y': np.float64(18.68), '1Y': np.float64(18.58), '2Y': np.float64(18.06), '3Y': np.float64(17.48), '5Y': np.float64(16.53), '7Y': np.float64(15.87), '10Y': np.float64(15.22), '15Y': np.float64(14.57), '20Y': np.float64(14.22), '30Y': np.float64(13.9)}}
2025-04-27 18:13:48 - KBDAnalyzer - INFO - Сформированы метрики КБД: {'date': Timestamp('2024-12-30 00:00:00'), 'market_state': 'inverted', 'optimal_duration': (0.1, 2.0), 'yield_threshold': np.float64(21.58), 'kbd_values': {'0.5Y': np.float64(18.75), '0.75Y': np.float64(18.68), '1Y': np.float64(18.58), '2Y': np.fl

КБД визуализация сохранена в: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/viz/kbd_curve_20250427_181348.png
Рекомендуемая стратегия взвешивания: equal - Равное взвешивание для защиты от инверсии кривой доходности

Параметры фильтрации облигаций:
  min_yield: 19.58
  max_yield: 46.58
  min_duration: 0.09000000000000001
  max_duration: 2.4749999999999996
  market_condition: inverted
  excluded_issuers: ['ВТБ', 'Мечел']
Обработано 4 облигаций

=== Рекомендованный облигационный портфель ===
     security_code   full_name  yield  duration_years  weight
6555  RU000A105EW9  Сегежа3P1R  21.33        0.558333    0.25
6547  RU000A0JSGV0  РЖД-32 обл  21.48        1.155833    0.25
6546  KG000A3LSJ06  Кыргыз. 02  20.85        0.677500    0.25
6548  RU000A0JUAH8  РЖД-30 обл  20.66        0.530833    0.25

Портфель сохранен в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/portfolio/bond_portfolio_20250427.csv
В

...


=== Характеристики портфеля ===
Средневзвешенная доходность: 21.08%
Средневзвешенная дюрация: 0.73 лет
Уровень риска (дюрация): Низкий

Анализ успешно завершен!

Недостаточная диверсификация. Корректировка параметров (итерация 2):
  Новая мин. доходность: 15.579999999999998
  Новый диапазон дюрации: 0.1-4.475
Обработано 4 облигаций

=== Рекомендованный облигационный портфель ===
     security_code   full_name  yield  duration_years  weight
6555  RU000A105EW9  Сегежа3P1R  21.33        0.558333    0.25
6547  RU000A0JSGV0  РЖД-32 обл  21.48        1.155833    0.25
6546  KG000A3LSJ06  Кыргыз. 02  20.85        0.677500    0.25
6548  RU000A0JUAH8  РЖД-30 обл  20.66        0.530833    0.25

Портфель сохранен в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/portfolio/bond_portfolio_20250427.csv


2025-04-27 18:13:50 - BondsProcessor - INFO - Visualization saved to /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/portfolio/bond_universe_20250427.png


Визуализация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/portfolio/bond_universe_20250427.png


2025-04-27 18:13:50 - BondsProcessor - INFO - Portfolio analysis generated successfully
2025-04-27 18:13:50 - BondsKBDPipeline - WARNING - Итерация 3: Сформировано только 4 облигаций (минимум 5)
2025-04-27 18:13:50 - BondsKBDPipeline - INFO - Скорректированные параметры для итерации 4: {'min_yield': np.float64(13.579999999999998), 'max_yield': np.float64(46.58), 'min_duration': 0.1, 'max_duration': 5.475, 'market_condition': 'inverted', 'excluded_issuers': ['ВТБ', 'Мечел']}
2025-04-27 18:13:50 - BondsKBDPipeline - INFO - Сводка результатов сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/results/bonds_results_20250427_181348.json
2025-04-27 18:13:50 - BondsKBDReport - INFO - Комбинированный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bonds/reports/bonds_kbd_report_20250427_181348.html
2025-04-27 18:13:50 - BondsKBDPipeline - INFO - Пайплайн выбора облигаций с учетом КБД завер


=== Характеристики портфеля ===
Средневзвешенная доходность: 21.08%
Средневзвешенная дюрация: 0.73 лет
Уровень риска (дюрация): Низкий

Анализ успешно завершен!

Недостаточная диверсификация. Корректировка параметров (итерация 3):
  Новая мин. доходность: 13.579999999999998
  Новый диапазон дюрации: 0.1-5.475

Итоговые параметры фильтрации:
  min_yield: 13.579999999999998
  max_yield: 46.58
  min_duration: 0.1
  max_duration: 5.475
  market_condition: inverted
  excluded_issuers: ['ВТБ', 'Мечел']


In [39]:
pipeline_results = executor.run_pipeline(
    # Основные параметры
    tickers_list=tickers_list,
    bond_results=bond_results,
    strategy_profile=pipeline_params['strategy_profile'],
    
    # Параметры для разных компонентов
    signal_params=signal_params,
    standard_portfolio_params=standard_portfolio_params,
    short_portfolio_params=short_portfolio_params,
    combined_portfolio_params=combined_portfolio_params,
    optimization_params=optimization_params,
    backtest_params=backtest_params,
    
    # Управление запускаемыми портфелями
    portfolio_controls={
        'run_standard_portfolio': True,
        'run_short_portfolio': pipeline_params['strategy_profile'] == 'aggressive',
        'run_combined_portfolio': pipeline_params['strategy_profile'] in ['moderate', 'aggressive'],
        'override_risk_profile': False
    },
    
    # Параметры выбора лучшего портфеля
    select_portfolio_params={
        'metrics_priority': ['sharpe', 'return', 'volatility'],
        'min_sharpe': 0,
        'prefer_standard': False
    },
    
    # Параметры для отчета и визуализации
    report_params={
        'include_charts': True,
        'include_metrics': True,
        'include_weights': True
    },
    visualization_params={
        'plot_style': 'ggplot',  # or 'dark_background', 'bmh', 'fivethirtyeight'
        'chart_size': (12, 8),
        'dpi': 300
    }
)


2025-04-27 18:15:53 - PipelineExecutor - INFO - Запуск пайплайна (ID: run_20250427_181346_balanced_portfolio_0427_1813, профиль: moderate)
2025-04-27 18:15:53 - PipelineExecutor - INFO - Скопирован портфель облигаций: /Users/aeshef/Documents/GitHub/kursach/data/pipeline_runs/run_20250427_181346_balanced_portfolio_0427_1813/bond_portfolio.csv
2025-04-27 18:15:53 - PipelineExecutor - INFO - Запуск генерации сигналов с параметрами: {'weight_tech': 0.5, 'weight_sentiment': 0.3, 'weight_fundamental': 0.2, 'threshold_buy': 0.5, 'threshold_sell': -0.5, 'top_pct': 0.3}
2025-04-27 18:15:53 - SignalGenerator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-27 18:15:53 - SignalGenerator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-27 18:15:53 - SignalGenerator - INFO - Загружено 7110 строк данных
2025-04-27 18:15:53 - SignalGenerator - INFO - Расчет композитного скора
2025-04-27 18:15:53 - SignalGenerator - INFO - Технический скор рассчитан н

<!-- я делаю курсовой проект. я придерживаюсь этого плана:
# ---  
## 1. Слой Сбора и Агрегации Данных

### 1.1. Рыночные данные
- Источники API: Yahoo Finance, Alpha Vantage для акций. Для облигаций/ОФЗ — выгрузки или ручная подготовка, если нет автосбора.
- Загрузка исторических котировок всех бумаг, которые потенциально могут войти в портфель (акции + risk-free/фонды).
- Сохранение единой “сырых” данных (CSV/Parquet).

### 1.2. Фундаментальные данные
- Используются только последние доступные отчёты по каждой бумаге (например, за 2023–24, чаще нет).
- Допускается ручной ввод/подготовка csv коллегой (дефолтно — только 1 snapshot на тикер).
- Промышленный подход: между апдейтами отчётов — “forward fill” фундаментальных коэффициентов; при вычислениях делается анализ только cross-sectional (по состоянию на одну дату между бумагами).
- Стандартизация (z-score) коэффициентов для последующего скоринга.

### 1.3. Новостной/сентимент-поток
- Автоматизация выгрузки новостей (RSS, Twitter, базовые каналы для акций и risk-free).
- Анализ новостного сентимента (проще — ручная классификация “важные макро/кризисы” для risk-free бумаг).
- Сохранение текстовых и сентиментных фич в отдельные поля.

---

## 2. Слой Предобработки и Признаков

### 2.1. Генерация признаков для акций и risk-free
- Чистка рядов; заполнение пропусков.
- Генерация технических индикаторов (SMA, EMA, RSI, MACD, Bollinger Bands, паттерны — минимум сигналы типа "пересечение SMA").
- Для risk-free инструментов — расчет доходностей/дюраций “на сейчас”, если есть хотя бы скриншот доходности.
- Для всех сегментов — доходности, волатильности, ковариации.

### 2.2. Работа с фундаменталом (ограничена snapshot-режимом!)
- Для единственного года: сравнение бумаг между собой — расчёт z-score фундаментальных метрик на дату.
- Проведение статического ранжирования (квантили, top/bottom группы).
- Генерация сводного композитного score для отбора universe.

### 2.3. Интеграция сентимента и фундаментала
- Корректировка статичного funda-score с учётом свежих новостных сигналов (например, большой рынок негатив — понизить вес funda-score).
- Для risk-free — влияние только “макро”-новостей и текущей ставочной среды.

---

## 3. Модуль анализа и сигналов

### 3.1. Генерация торговых сигналов
- Акции: Buy/Hold/Sell по композитному score (технические + функдаментал + новости).
- Для risk-free: всегда Hold/Buy (core allocation), исключения — при негативных макро/ставках.

### 3.2. Кросс-секционный квантильный отбор
- “Shortlist” бумаг для портфеля: top 30–50% по funda-score, приоритизированных техническими индикаторами.
- Для risk-free инструментов можно просто фиксировать веса или задавать диапазон для оптимизации.

---  

## 4. Оптимизация Портфеля

### 4.1. Модель Марковица (Mean-Variance)
- Оптимизация среди shortlist бумаг (только Buy/Hold).
- Явно фиксируем вес на risk-free (“core allocation”, например 25–35%).
- Остальное распределяем по акциям оптимально по риск-доходности (mean-variance).

### 4.2. Black-Litterman (опционально)
- Возможность внести “view” на основе ранга funda-score и новостных сигналов.
- Коррекция “базовых” рыночных весов с помощью своих score.

---

## 5. Интеграция, отчётность и автоматизация

### 5.1. Финальные сигналы и обработка портфеля
- Взвешенная агрегация сигналов (funda-score × техника × новости).
- Единый пайплайн: загрузка — признаки — сигналы — оптимизация — результат.
- Пайплайны собраны как большие скрипты-автоматизаторы для (квази)ежедневного обновления.

### 5.2. Интерфейс пользователя
- Просмотр результатов
по кнопке (CLI/Jupyter/Telegram-бот).
- Возможность быстро изменить параметры universe/core allocation.

---

## 6. Бэктестинг и визуализация

### 6.1. Бэктест
- Проверка стратегии на годичном горизонте (по доступным данным).
- Особое внимание: для фундаментала — тестирование, как бы работали “портфели top vs bottom квантилей по snapshot”.
- Акцент на анализ разницы “просто держать весь рынок” vs. “отбирать по фунду/score”.

### 6.2. Визуализация
- Графики динамики портфеля, групп, эффективная граница.
- Визуализация сравнения групп, влияния сигналов.

---

## 7. Автоматизация и пайплайны

- Все ключевые этапы реализованы как отдельные большие py-скрипты (классы: DataCollector, FeatureEngineer, SignalGenerator, PortfolioOptimizer, Backtester, Visualizer).
- Каждый пайплайн сохраняет промежуточные дампы для отладки.
- Финальный мастер-пайплайн (“одна кнопка”).
- Делегирование ручных элементов коллегой: подготовка funda/csv, ручная разметка событий/новостей, первичный запуск функций.

---

## 8. Особенности и работа с мало меняющимися фундаментальными данными

- Прямо указывается в отчёте: фундаментальные признаки “медленные”, обновляются редко, анализ кросс-секционный: используется только сравнение между бумагами на одну дату, либо квантилизация и ранжирование при каждом отчёте.
- Расширение базы данных даже на 2–3 года (если возможно) — значительно повысит качество анализа и возможности rolling backtest.
- Все краткосрочные сигналы (техникa, новости) агрегируются для “ускорения” pipeline решений между публикациями отчетов.

---

## 9. Как преподнести в работе/отчёте

> Фундаментальные данные используются в cross-sectional анализе и квантилизации портфелей из-за своей низкой частоты обновления и ограниченной истории. Это соответствует практике индустрии для развивающихся рынков/ограниченных источников. При появлении новых отчетов pipeline автоматически обновляет ранжирование бумаг и состава портфеля. Краткосрочные решения строятся преимущественно на технических и новостных сигналах.

---

## Если понадобится — скину шаблоны py-файлов и классов по этому плану.


на данный момент я нахожусь на таком этапе: 
1) структура проекта
/Users/aeshef/Documents/GitHub/kursach
(├── df.csv - обьединенный датасет с 384 фичами для 17 тикеров за 2024-2025 год
├── figi.txt
├── news_analysis_summary.txt
├── processed_data
│   ├── AFKS
│   │   ├── AFKS.parquet
│   │   ├── AFKS_all_news_processed.csv
│   │   ├── AFKS_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── AFKS_daily_events.csv
│   │   │   ├── AFKS_daily_sentiment.csv
│   │   │   ├── AFKS_event_distribution.png
│   │   │   ├── AFKS_ml_features.csv
│   │   │   ├── AFKS_news_with_events.csv
│   │   │   ├── AFKS_news_with_sentiment.csv
│   │   │   ├── AFKS_price_vs_sentiment.png
│   │   │   ├── AFKS_sentiment_distribution.png
│   │   │   ├── AFKS_sentiment_dynamics.png
│   │   │   ├── AFKS_sentiment_features.csv
│   │   │   └── AFKS_topics.txt
│   │   └── tech_analysis
│   │       ├── AFKS_MACD.png
│   │       ├── AFKS_price_indicators.png
│   │       └── AFKS_tech_indicators.csv
...
│   ├── VTBR
│   │   ├── VTBR.parquet
│   │   ├── VTBR_all_news_processed.csv
│   │   ├── VTBR_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── VTBR_daily_events.csv
│   │   │   ├── VTBR_daily_sentiment.csv
│   │   │   ├── VTBR_event_distribution.png
│   │   │   ├── VTBR_ml_features.csv
│   │   │   ├── VTBR_news_with_events.csv
│   │   │   ├── VTBR_news_with_sentiment.csv
│   │   │   ├── VTBR_price_vs_sentiment.png
│   │   │   ├── VTBR_sentiment_distribution.png
│   │   │   ├── VTBR_sentiment_dynamics.png
│   │   │   ├── VTBR_sentiment_features.csv
│   │   │   └── VTBR_topics.txt
│   │   └── tech_analysis
│   │       ├── VTBR_MACD.png
│   │       ├── VTBR_price_indicators.png
│   │       └── VTBR_tech_indicators.csv
│   ├── bonds_csv
│   │   ├── bond_search_2023-12-01.csv
│   │   ├── bond_search_2024-03-26.csv
│   │   ├── bond_search_2024-04-10.csv
│   │   ├── bond_search_2024-05-24.csv
│   │   ├── bond_search_2024-08-09.csv
│   │   ├── bond_search_2024-08-10.csv
│   │   ├── bond_search_2024-08-12.csv
│   │   ├── bond_search_2024-09-04.csv
│   │   ├── bond_search_2024-09-13.csv
│   │   ├── bond_search_2024-10-30.csv
│   │   ├── bond_search_2024-11-26.csv
│   │   ├── bond_search_2024-12-04.csv
│   │   ├── bond_search_2025-01-28.csv
│   │   ├── bond_search_2025-02-10.csv
│   │   ├── bond_search_2025-02-14.csv
│   │   └── bond_search_2025-04-22.csv
...
├── raw_data
│   ├── AFKS
│   │   ├── BBG004S68614_2024
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240102.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240103.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240104.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240105.csv
...

├── status.json
├── telegram_news
│   ├── AFKS_telegram_news_20250422.csv
│   ├── ALRS_telegram_news_20250422.csv
│   ├── GAZP_telegram_news_20250422.csv
│   ├── GMKN_telegram_news_20250422.csv
│   ├── LKOH_telegram_news_20250422.csv
│   ├── MAGN_telegram_news_20250422.csv
│   ├── MTSS_telegram_news_20250422.csv
│   ├── MVID_telegram_news_20250422.csv
│   ├── NVTK_telegram_news_20250422.csv
│   ├── PHOR_telegram_news_20250422.csv
│   ├── ROSN_telegram_news_20250422.csv
│   ├── RUAL_telegram_news_20250422.csv
│   ├── SBER_telegram_news_20250422.csv
│   ├── SIBN_telegram_news_20250422.csv
│   ├── SNGS_telegram_news_20250422.csv
│   ├── TATN_telegram_news_20250422.csv
│   ├── VTBR_telegram_news_20250422.csv
│   └── telegram_headlines_20250422_160151.csv
└── tickers.csv) - конец папки data
├── ipynbs
│   ├── market_data.log
│   └── overall_data_pipeline.ipynb - тут в виде ноутбука (потому будет py файлом) расписан полный пайплайн сбора тикеров, тех анализа, фундаментального анализа, извлечения фичей из новостей и получение объединенного датасета со всеми фичами для всех тикеров (df.csv)
├── news_pipeline_config.json
├── pys
│   ├── __pycache__
│   │   ├── event_detector.cpython-313.pyc
│   │   ├── news_feature_extractor.cpython-313.pyc
│   │   ├── news_integration.cpython-313.pyc
│   │   ├── news_preprocessor.cpython-313.pyc
│   │   ├── news_visualizer.cpython-313.pyc
│   │   └── sentiment_analysis.cpython-313.pyc
│   ├── data_integration.py
│   ├── event_detector.py
│   ├── fundamental_data.py
│   ├── market_data.py
│   ├── news_feature_extractor.py
│   ├── news_integration.py
│   ├── news_pipeline.py
│   ├── news_preprocessor.py
│   ├── news_visualizer.py
│   ├── portfolio_optimizer.py
│   ├── run_with_config.py
│   ├── sentiment_analysis.py
│   ├── tech_analysis.py
│   └── technical_indicators.py
└── telegram_session.session

2) я еще вообще никак не работал с данными bonds, так что с ними что-то надо сделать. 

у меня есть 7 дней на выполнение всего курсового проекта, так что сейчас от тебя мне нужно много кода. предложи в соответствии с планом, что мне стоит сейчас сделать -->